In [1]:
import subprocess
import json
import os
from bit import PrivateKeyTestnet
import web3
from web3 import Web3
from web3.middleware import geth_poa_middleware
from eth_account import Account
from dotenv import load_dotenv
load_dotenv()

from constants import *
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

Consider installing rusty-rlp to improve pyrlp performance with a rust based backend


In [2]:
from constants import *
mnemonic = os.getenv('MNEMONIC')

In [3]:
def derive_wallets(coin):
    mnemonic = os.getenv('MNEMONIC')

    #ln does not work for me, replaced with actual php instead
    command = f'php hd-wallet-derive\hd-wallet-derive.php -g --mnemonic="{mnemonic}" --cols=path,address,privkey,pubkey --format=json --coin={coin}'

    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()

    keys = json.loads(output)
    return keys

def priv_key_to_account(coin, priv_key):
    if (coin == 'eth'):
        return web3.eth.Account.privateKeyToAccount(priv_key)
    elif (coin == 'btc-test'):
        return PrivateKeyTestnet(priv_key)
    else:
        return None

def create_tx(coin , account, to, amount):
    if (coin == 'eth'):
        return {
            'to': to,
            'from': account,
            'value': amount,
            'gas': w3.eth.estimateGas({
                "from": account.address,
                "to": to,
                "value": amount
            }),
            'gasPrice': w3.eth.gasPrice,
            'nonce': w3.eth.getTransactionCount(account.address),
            'chainID': coins[coin][0]['path'].split("'/")[1]
        }
    elif (coin == 'btc-test'):
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])
    else:
        return None

In [4]:
coin_list = [BTC, ETH, BTCTEST]
coins = {}
for coin in coin_list:
    coins[coin] = derive_wallets(coin)

In [6]:
coins[BTCTEST]

[{'path': "m/44'/1'/0'/0/0",
  'address': 'mrEpvqTHeU52Aj5SRsux33V4xwqEmWcbAW',
  'privkey': 'cTvT1LAHD9X3BLiKRJbrtCkErMeRaJwpDrNtZCAQw9MGWycJ2pXf',
  'pubkey': '02c15d8c1d85ca5db116fc6ad7e8c48c3a2be7b3f9877d02069322a52b654dfff7'},
 {'path': "m/44'/1'/0'/0/1",
  'address': 'mpJbDpsFBtu8wDJDsLuHBcGbnf3T2BhEQL',
  'privkey': 'cQbwVLXcXjH75vAnTr4CjFriqmn7yXPzuCxvvMeFjJ91ywES8rM1',
  'pubkey': '03fc68ca7877ee3caeb3febcb43b1d563f39fbbbab333235a758cf9389d43a905c'},
 {'path': "m/44'/1'/0'/0/2",
  'address': 'miRZq1aZpGM4aztZdAPezmUEtNP8f6fwQD',
  'privkey': 'cVt5KTfpsrnKUiMYeqeVequYr5jXby5xnoToMtPzky84rk54UHxR',
  'pubkey': '03a46b7afa200f696edd20e1f93a0cd0ad9b1f484f8062ef0bc5bb420404c1f6dc'},
 {'path': "m/44'/1'/0'/0/3",
  'address': 'mrnKe5nJHmZjcJURvHoa5ptiPTAuRoTzwg',
  'privkey': 'cT6R7RN89QT13LZKVagCD6uz2Y8ZBoe18m8E3HkB9dFA1LFSShCc',
  'pubkey': '030cf5cf0885a4f67688d2943ab996b24d5f30e0bc31de25ff5f969edac8f170b9'},
 {'path': "m/44'/1'/0'/0/4",
  'address': 'msQ2is4R3wNuHpA2AVQNYgK79Kiaf

In [124]:
raw_tx = create_tx(ETH, priv_key_to_account(ETH, coins[ETH][0]['privkey']), coins[ETH][1]['address'], 1)
raw_tx
# dir(w3.eth)

{'to': '0x8f025C633D9548B08637A6Fe8b67e91F1d59F1a9',
 'from': <eth_account.signers.local.LocalAccount at 0x2a7046edb48>,
 'value': 1,
 'gas': 21000,
 'gasPrice': 20000000000,
 'nonce': 0,
 'chainID': '60'}

In [ ]:
web3.eth.Account.sign_transaction(raw_tx, private_key=coins[ETH][0]['privkey'])